<a href="https://colab.research.google.com/github/Dazhou2000/Msc-Project/blob/master/SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!nvidia-smi

Tue Jul 23 12:11:23 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   42C    P8              12W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# This get the RAPIDS-Colab install files and test check your GPU.  Run this and the next cell only.
# Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py


Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 490, done.
remote: Counting objects: 100% (221/221), done.
remote: Compressing objects: 100% (130/130), done.
remote: Total 490 (delta 149), reused 124 (delta 91), pack-reused 269
Receiving objects: 100% (490/490), 136.70 KiB | 777.00 KiB/s, done.
Resolving deltas: 100% (251/251), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 1.7 MB/s eta 0:00:00
Installing the rest of the RAPIDS 24.4.* libraries
Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 GB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 GB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.8/137.8 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.9/920.9 kB 61.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import cuml
cuml.__version__

'24.04.00'

In [ ]:
import locale
import os
import subprocess

# Set the locale within Python
os.environ['LC_ALL'] = 'C.UTF-8'
os.environ['LANG'] = 'C.UTF-8'
locale.setlocale(locale.LC_ALL, 'C.UTF-8')

# Install cuML and Optuna using subprocess
try:
    subprocess.run(['pip', 'install', 'optuna'], check=True)
except subprocess.CalledProcessError as e:
    print("Error during installation:", e)


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from imblearn.over_sampling import SMOTE
from cuml.svm import SVC
from cuml.metrics import roc_auc_score
from sklearn.metrics import classification_report, accuracy_score
import optuna
import json
import os

def load_data(patient_id, window_size):
    data_path = f'/content/drive/My Drive/Msc Project/features_and_labels_detection/P{patient_id} features and seizure labels with window_{window_size}.csv'
    data = pd.read_csv(data_path)
    data = data.dropna()
    importance = pd.read_csv(f'/content/drive/My Drive/Msc Project/importance_feature/P{patient_id}MDI_importance with window_{window_size}.csv')
    top_features_list = importance.sort_values(by='Importance', ascending=False)['Unnamed: 0'].tolist()
    return data, top_features_list

def save_best_params(patient_id, window_size, best_params):
    if not os.path.exists(f'/content/drive/My Drive/Msc Project/best_params_svm'):
        os.makedirs(f'/content/drive/My Drive/Msc Project/best_params_svm')
    params_file = f'/content/drive/My Drive/Msc Project/best_params_svm/params_patient_{patient_id}_window_{window_size}.json'
    with open(params_file, 'w') as file:
        json.dump(best_params, file)

def load_best_params(patient_id, window_size):
    params_file = f'/content/drive/My Drive/Msc Project/best_params_svm/params_patient_{patient_id}_window_{window_size}.json'
    if os.path.exists(params_file):
        with open(params_file, 'r') as file:
            best_params = json.load(file)
        return best_params
    else:
        return None

def save_best_scores(patient_id, window_size, best_scores):
    if not os.path.exists(f'/content/drive/My Drive/Msc Project/best_scores_svm'):
        os.makedirs(f'/content/drive/My Drive/Msc Project/best_scores_svm')
    scores_file = f'/content/drive/My Drive/Msc Project/best_scores_svm/best_scores_more.csv'
    row_name = f'{patient_id}_window_{window_size}'
    scores_df = pd.DataFrame(best_scores, index=[row_name])
    if os.path.exists(scores_file):
        existing_df = pd.read_csv(scores_file, index_col=0)
        if row_name in existing_df.index:
            existing_df.loc[row_name] = scores_df.loc[row_name]
            existing_df.to_csv(scores_file)
        else:
            updated_df = pd.concat([existing_df, scores_df])
            updated_df.to_csv(scores_file)
    else:
        scores_df.to_csv(scores_file)

def run_optimization(patient_id, window_size):
    data, top_features_list = load_data(patient_id, window_size)
    selected_features = pd.read_csv(f'/content/drive/My Drive/Msc Project/select_feature/P{patient_id}_selected_features_with_window_{window_size}.csv')
    number_features = selected_features.loc[selected_features['AUC'].idxmax()]['Num Features']
    X = data.drop(columns=['label', 'timestamp', 'seizure_event', 'Unnamed: 0'])
    X = data[top_features_list[:int(number_features)]]
    y = data['label']

    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, shuffle=False, random_state=42)
    smote = SMOTE(random_state=42, n_jobs=-1)
    X_train, y_train = smote.fit_resample(X_train, y_train)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, shuffle=False, random_state=42)

    def objective(trial):
        params = {
            'C': trial.suggest_float('C', 1e-5, 1e0, log=True),
            'gamma': trial.suggest_float('gamma', 1e-5, 1e0, log=True),
            'kernel': 'rbf'
        }
        svm_model = SVC(**params, cache_size=10000, class_weight='balanced', random_state=42)
        svm_model.fit(X_train, y_train)
        y_pred_val_scores = svm_model.decision_function(X_val)
        roc_auc = roc_auc_score(y_val, y_pred_val_scores)
        return roc_auc

    class EarlyStoppingCallback:
        def __init__(self, patience: int):
            self.patience = patience
            self.best_value = None
            self.no_improvement_count = 0

        def __call__(self, study, trial):
            if self.best_value is None or study.best_value > self.best_value:
                self.best_value = study.best_value
                self.no_improvement_count = 0
            else:
                self.no_improvement_count += 1
            if self.no_improvement_count >= self.patience:
                study.stop()

    study = optuna.create_study(direction='maximize')
    best_params = load_best_params(patient_id, window_size)
    if best_params:
        study.enqueue_trial(best_params)

    early_stopping = EarlyStoppingCallback(patience=50)
    study.optimize(objective, n_trials=200, callbacks=[early_stopping])

    best_params = study.best_params
    save_best_params(patient_id, window_size, best_params)

    svm_model = SVC(**best_params, kernel='rbf', cache_size=10000, class_weight='balanced', random_state=42)
    svm_model.fit(X_train, y_train)

    y_pred_train = svm_model.predict(X_train)
    y_pred_val = svm_model.predict(X_val)
    y_pred_test = svm_model.predict(X_test)

    y_pred_train_scores = svm_model.decision_function(X_train)
    auc_train = roc_auc_score(y_train, y_pred_train_scores)
    y_pred_val_scores = svm_model.decision_function(X_val)
    auc_val = roc_auc_score(y_val, y_pred_val_scores)
    y_pred_test_scores = svm_model.decision_function(X_test)
    auc_test = roc_auc_score(y_test, y_pred_test_scores)

    print("Train classification report:")
    print(classification_report(y_train, y_pred_train))

    print("Validation classification report:")
    print(classification_report(y_val, y_pred_val))

    print("Test classification report:")
    print(classification_report(y_test, y_pred_test))

    print("Train accuracy:", accuracy_score(y_train, y_pred_train))
    print("Validation accuracy:", accuracy_score(y_val, y_pred_val))
    print("Test accuracy:", accuracy_score(y_test, y_pred_test))

    print("Train AUC:", auc_train)
    print("Validation AUC:", auc_val)
    print("Test AUC:", auc_test)

    best_scores = {
        "sensitivity": classification_report(y_test, y_pred_test, output_dict=True)['1']['recall'],
        "auc": auc_test,
        "specificity": classification_report(y_test, y_pred_test, output_dict=True)['0']['recall'],
        "accuracy": accuracy_score(y_test, y_pred_test)
    }
    save_best_scores(patient_id, window_size, best_scores)

patient_ids = ['1_', '3_', '4_', '10', '13', '19', '23', '27', '29', '30', '31']
window_sizes = [24, 72, 144, 288]

for patient_id in patient_ids:
    for window_size in window_sizes:
        run_optimization(patient_id, window_size)


/usr/local/lib/python3.10/dist-packages/imblearn/over_sampling/_smote/base.py:336: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(
[I 2024-07-17 16:19:12,504] A new study created in memory with name: no-name-d6ecd1f6-9dfd-4310-b82c-e33791ff0ae7
[I 2024-07-17 16:19:50,285] Trial 0 finished with value: 0.8064193725585938 and parameters: {'C': 0.061505822213405864, 'gamma': 0.0019190194901119498}. Best is trial 0 with value: 0.8064193725585938.


Train classification report:
              precision    recall  f1-score   support

           0       0.73      0.57      0.64    112280
           1       0.65      0.79      0.71    112280

    accuracy                           0.68    224560
   macro avg       0.69      0.68      0.68    224560
weighted avg       0.69      0.68      0.68    224560

Validation classification report:
              precision    recall  f1-score   support

           0       1.00      0.58      0.73     13993
           1       0.04      0.93      0.08       288

    accuracy                           0.58     14281
   macro avg       0.52      0.75      0.41     14281
weighted avg       0.98      0.58      0.72     14281

Test classification report:
              precision    recall  f1-score   support

           0       0.99      0.63      0.77     14079
           1       0.03      0.74      0.05       203

    accuracy                           0.63     14282
   macro avg       0.51      0.69    

/usr/local/lib/python3.10/dist-packages/imblearn/over_sampling/_smote/base.py:336: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(
[I 2024-07-17 16:20:55,413] A new study created in memory with name: no-name-f22a7202-b252-4cf3-80ee-87a5fc8a6aeb
[I 2024-07-17 16:21:20,898] Trial 0 finished with value: 0.8494018316268921 and parameters: {'C': 0.011458637348403897, 'gamma': 0.010542785507952639}. Best is trial 0 with value: 0.8494018316268921.


Train classification report:
              precision    recall  f1-score   support

           0       0.72      0.63      0.67    108671
           1       0.67      0.76      0.71    108671

    accuracy                           0.69    217342
   macro avg       0.70      0.69      0.69    217342
weighted avg       0.70      0.69      0.69    217342

Validation classification report:
              precision    recall  f1-score   support

           0       0.99      0.64      0.78     13494
           1       0.13      0.91      0.22       783

    accuracy                           0.65     14277
   macro avg       0.56      0.77      0.50     14277
weighted avg       0.94      0.65      0.75     14277

Test classification report:
              precision    recall  f1-score   support

           0       0.99      0.67      0.80     13786
           1       0.08      0.78      0.14       491

    accuracy                           0.68     14277
   macro avg       0.53      0.72    

/usr/local/lib/python3.10/dist-packages/imblearn/over_sampling/_smote/base.py:336: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(
[I 2024-07-17 16:22:27,137] A new study created in memory with name: no-name-1513b1d0-9f43-4085-b010-1ef07b13078b
[I 2024-07-17 16:22:48,736] Trial 0 finished with value: 0.7883658409118652 and parameters: {'C': 4.5327867611169305e-05, 'gamma': 0.2048392591563504}. Best is trial 0 with value: 0.7883658409118652.


...

Train classification report:
              precision    recall  f1-score   support

           0       0.73      0.84      0.78    101979
           1       0.81      0.69      0.75    101979

    accuracy                           0.77    203958
   macro avg       0.77      0.77      0.76    203958
weighted avg       0.77      0.77      0.76    203958

Validation classification report:
              precision    recall  f1-score   support

           0       0.93      0.86      0.89     12578
           1       0.32      0.48      0.39      1664

    accuracy                           0.82     14242
   macro avg       0.62      0.67      0.64     14242
weighted avg       0.86      0.82      0.84     14242

Test classification report:
              precision    recall  f1-score   support

           0       0.96      0.90      0.93     13511
           1       0.14      0.30      0.19       732

    accuracy                           0.87     14243
   macro avg       0.55      0.60    

/usr/local/lib/python3.10/dist-packages/imblearn/over_sampling/_smote/base.py:336: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(
[I 2024-07-17 17:05:30,693] A new study created in memory with name: no-name-a559875e-c5a8-4287-a437-bdd7c58dd8f4
[I 2024-07-17 17:05:48,427] Trial 0 finished with value: 0.7371134757995605 and parameters: {'C': 0.0043487615922041456, 'gamma': 0.05646353271973432}. Best is trial 0 with value: 0.7371134757995605.


Train classification report:
              precision    recall  f1-score   support

           0       0.74      0.83      0.78     94256
           1       0.80      0.71      0.75     94256

    accuracy                           0.77    188512
   macro avg       0.77      0.77      0.77    188512
weighted avg       0.77      0.77      0.77    188512

Validation classification report:
              precision    recall  f1-score   support

           0       0.87      0.85      0.86     11407
           1       0.45      0.49      0.47      2832

    accuracy                           0.78     14239
   macro avg       0.66      0.67      0.66     14239
weighted avg       0.79      0.78      0.78     14239

Test classification report:
              precision    recall  f1-score   support

           0       0.92      0.89      0.91     13027
           1       0.15      0.21      0.18      1212

    accuracy                           0.83     14239
   macro avg       0.54      0.55    

/usr/local/lib/python3.10/dist-packages/imblearn/over_sampling/_smote/base.py:336: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(
[I 2024-07-17 17:06:40,592] A new study created in memory with name: no-name-495d6994-13ff-4bbb-9661-26d918b80771
[I 2024-07-17 17:06:56,685] Trial 0 finished with value: 0.7264726161956787 and parameters: {'C': 0.018096528799059687, 'gamma': 0.0005317738265257927}. Best is trial 0 with value: 0.7264726161956787.


Train classification report:
              precision    recall  f1-score   support

           0       0.64      0.81      0.72     81314
           1       0.74      0.55      0.63     81314

    accuracy                           0.68    162628
   macro avg       0.69      0.68      0.67    162628
weighted avg       0.69      0.68      0.67    162628

Validation classification report:
              precision    recall  f1-score   support

           0       0.76      0.92      0.83      9623
           1       0.69      0.38      0.49      4602

    accuracy                           0.74     14225
   macro avg       0.72      0.65      0.66     14225
weighted avg       0.73      0.74      0.72     14225

Test classification report:
              precision    recall  f1-score   support

           0       0.78      0.15      0.26     12155
           1       0.13      0.74      0.22      2070

    accuracy                           0.24     14225
   macro avg       0.45      0.45    

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from imblearn.over_sampling import SMOTE
from cuml.svm import SVC
from cuml.metrics import roc_auc_score
from sklearn.metrics import classification_report, accuracy_score
import optuna
import json
import os

SEED = 42
np.random.seed(SEED)

def load_data(patient_id, window_size, forecasting_length):
    data_path = f'/content/drive/My Drive/Msc Project/features_and_labels_detection/P{patient_id} features and seizure labels with window_{window_size}.csv'
    data = pd.read_csv(data_path)
    data = data.dropna()
    importance = pd.read_csv(f'/content/drive/My Drive/Msc Project/Forecasting/importance_feature/P{patient_id}MDI_importance with window_{window_size} with forecasting_{forecasting_length}.csv')
    top_features_list = importance.sort_values(by='Importance', ascending=False)['Unnamed: 0'].tolist()
    return data, top_features_list

def save_best_params(patient_id, window_size, forecasting_length, best_params):
    if not os.path.exists(f'/content/drive/My Drive/Msc Project/Forecasting/best_params_svm'):
        os.makedirs(f'/content/drive/My Drive/Msc Project/Forecasting/best_params_svm')
    params_file = f'/content/drive/My Drive/Msc Project/Forecasting/best_params_svm/params_patient_{patient_id}_window_{window_size}_forecasting_{forecasting_length}.json'
    with open(params_file, 'w') as file:
        json.dump(best_params, file)

def load_best_params(patient_id, window_size, forecasting_length):
    params_file = f'/content/drive/My Drive/Msc Project/Forecasting/best_params_svm/params_patient_{patient_id}_window_{window_size}_forecasting_{forecasting_length}.json'
    if os.path.exists(params_file):
        with open(params_file, 'r') as file:
            best_params = json.load(file)
        return best_params
    else:
        return None

def save_best_scores(patient_id, window_size, forecasting_length, best_scores):
    if not os.path.exists(f'/content/drive/My Drive/Msc Project/Forecasting/best_scores_svm'):
        os.makedirs(f'/content/drive/My Drive/Msc Project/Forecasting/best_scores_svm')
    scores_file = f'/content/drive/My Drive/Msc Project/Forecasting/best_scores_svm/best_scores.csv'
    row_name = f'{patient_id}_window_{window_size}_forecasting_{forecasting_length}'
    scores_df = pd.DataFrame(best_scores, index=[row_name])
    if os.path.exists(scores_file):
        existing_df = pd.read_csv(scores_file, index_col=0)
        if row_name in existing_df.index:
            existing_df.loc[row_name] = scores_df.loc[row_name]
            existing_df.to_csv(scores_file)
        else:
            updated_df = pd.concat([existing_df, scores_df])
            updated_df.to_csv(scores_file)
    else:
        scores_df.to_csv(scores_file)

def process_patient_window(patient_id, window_size, forecasting_length):
    data, top_features_list = load_data(patient_id, window_size, forecasting_length)

    selected_features = pd.read_csv(f'/content/drive/My Drive/Msc Project/Forecasting/select_feature/P{patient_id}_selected_features_with_window_{window_size}_with_forecasting_{forecasting_length}.csv')
    number_features = selected_features.loc[selected_features['AUC'].idxmax()]['Num Features']
    print(f'Number of features for patient {patient_id} with window {window_size}: {number_features}')

    X = data.drop(columns=['label', 'timestamp', 'seizure_event', 'Unnamed: 0'])
    X = X[top_features_list[:int(number_features)]]
    y = data['label']
    y = y.shift(-forecasting_length)

    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, shuffle=False, random_state=42)
    smote = SMOTE(random_state=42, n_jobs=-1)
    X_train, y_train = smote.fit_resample(X_train, y_train)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, shuffle=False, random_state=42)
    print(len(X_test))
    print(len(y_test))
    X_test = X_test.iloc[:-forecasting_length]
    y_test = y_test.iloc[:-forecasting_length]
    print(len(X_test))
    print(len(y_test))

    def objective(trial):
        params = {
            'C': trial.suggest_float('C', 1e-5, 1e0, log=True),
            'gamma': trial.suggest_float('gamma', 1e-5, 1e0, log=True),
            'kernel': 'rbf'
        }
        svm_model = SVC(**params, cache_size=10000, class_weight='balanced', random_state=42)
        svm_model.fit(X_train, y_train)
        y_pred_val_scores = svm_model.decision_function(X_val)
        roc_auc = roc_auc_score(y_val, y_pred_val_scores)
        return roc_auc

    class EarlyStoppingCallback:
        def __init__(self, patience: int):
            self.patience = patience
            self.best_value = None
            self.no_improvement_count = 0

        def __call__(self, study, trial):
            if self.best_value is None or study.best_value > self.best_value:
                self.best_value = study.best_value
                self.no_improvement_count = 0
            else:
                self.no_improvement_count += 1
            if self.no_improvement_count >= self.patience:
                study.stop()

    study = optuna.create_study(direction='maximize',sampler=optuna.samplers.TPESampler(seed=SEED))
    best_params = load_best_params(patient_id, window_size, forecasting_length)
    if best_params:
        study.enqueue_trial(best_params)

    early_stopping = EarlyStoppingCallback(patience=50)
    study.optimize(objective, n_trials=200, callbacks=[early_stopping])

    best_params = study.best_params
    print("Best parameters found by Optuna:", best_params)

    save_best_params(patient_id, window_size, forecasting_length, best_params)

    svm_model = SVC(**best_params, kernel='rbf', cache_size=10000, class_weight='balanced', random_state=42)
    svm_model.fit(X_train, y_train)

    y_pred_train = svm_model.predict(X_train)
    y_pred_val = svm_model.predict(X_val)
    y_pred_test = svm_model.predict(X_test)

    y_pred_train_scores = svm_model.decision_function(X_train)
    auc_train = roc_auc_score(y_train, y_pred_train_scores)
    y_pred_val_scores = svm_model.decision_function(X_val)
    auc_val = roc_auc_score(y_val, y_pred_val_scores)
    y_pred_test_scores = svm_model.decision_function(X_test)
    auc_test = roc_auc_score(y_test, y_pred_test_scores)

    print(f"Train classification report for patient {patient_id} with window {window_size}:")
    print(classification_report(y_train, y_pred_train))

    print(f"Validation classification report for patient {patient_id} with window {window_size}:")
    print(classification_report(y_val, y_pred_val))

    print(f"Test classification report for patient {patient_id} with window {window_size}:")
    print(classification_report(y_test, y_pred_test))

    print(f"Train accuracy for patient {patient_id} with window {window_size}: {accuracy_score(y_train, y_pred_train)}")
    print(f"Validation accuracy for patient {patient_id} with window {window_size}: {accuracy_score(y_val, y_pred_val)}")
    print(f"Test accuracy for patient {patient_id} with window {window_size}: {accuracy_score(y_test, y_pred_test)}")

    print(f"Train AUC for patient {patient_id} with window {window_size}: {auc_train}")
    print(f"Validation AUC for patient {patient_id} with window {window_size}: {auc_val}")
    print(f"Test AUC for patient {patient_id} with window {window_size}: {auc_test}")

    best_scores = {
        "auc": auc_test,
        "sensitivity": classification_report(y_test, y_pred_test, output_dict=True)['1.0']['recall'],
        "specificity": classification_report(y_test, y_pred_test, output_dict=True)['0.0']['recall'],
        "accuracy": accuracy_score(y_test, y_pred_test)
    }

    save_best_scores(patient_id, window_size, forecasting_length, best_scores)

patient_ids = ['27','30']
window_sizes = [288]
forecasting_lengths = [24,120]

for patient_id in patient_ids:
    for window_size in window_sizes:
        for forecasting_length in forecasting_lengths:
            print(f"Processing patient {patient_id} with window size {window_size} with forecasting length {forecasting_length}")
            process_patient_window(patient_id, window_size, forecasting_length)


Processing patient 27 with window size 288 with forecasting length 24
Number of features for patient 27 with window 288: 19.0


/usr/local/lib/python3.10/dist-packages/imblearn/over_sampling/_smote/base.py:336: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(
[I 2024-07-23 13:25:13,062] A new study created in memory with name: no-name-6cbd9931-ae3f-42d3-a45c-bb002f2f1d4c


11722
11722
11698
11698


[I 2024-07-23 13:25:20,494] Trial 0 finished with value: 0.5203731060028076 and parameters: {'C': 0.0007459343285726547, 'gamma': 0.566984951147885}. Best is trial 0 with value: 0.5203731060028076.
[I 2024-07-23 13:25:28,112] Trial 1 finished with value: 0.5422036051750183 and parameters: {'C': 0.0457056309980145, 'gamma': 0.009846738873614562}. Best is trial 1 with value: 0.5422036051750183.
[I 2024-07-23 13:25:35,687] Trial 2 finished with value: 0.5717393159866333 and parameters: {'C': 6.0268891286825045e-05, 'gamma': 6.025215736203858e-05}. Best is trial 2 with value: 0.5717393159866333.
[I 2024-07-23 13:25:43,287] Trial 3 finished with value: 0.5239570736885071 and parameters: {'C': 1.951722464144947e-05, 'gamma': 0.21423021757741043}. Best is trial 2 with value: 0.5717393159866333.
[I 2024-07-23 13:25:50,549] Trial 4 finished with value: 0.5362225770950317 and parameters: {'C': 0.010129197956845729, 'gamma': 0.034702669886504146}. Best is trial 2 with value: 0.5717393159866333.
[

Best parameters found by Optuna: {'C': 0.01946940993973371, 'gamma': 1.0015441744603262e-05}
Train classification report for patient 27 with window 288:
              precision    recall  f1-score   support

         0.0       0.53      0.89      0.66     64962
         1.0       0.65      0.20      0.30     64962

    accuracy                           0.55    129924
   macro avg       0.59      0.55      0.48    129924
weighted avg       0.59      0.55      0.48    129924

Validation classification report for patient 27 with window 288:
              precision    recall  f1-score   support

         0.0       0.62      0.86      0.72      7054
         1.0       0.49      0.20      0.29      4668

    accuracy                           0.60     11722
   macro avg       0.56      0.53      0.50     11722
weighted avg       0.57      0.60      0.55     11722

Test classification report for patient 27 with window 288:
              precision    recall  f1-score   support

         0.0  

...

Best parameters found by Optuna: {'C': 0.9974273234964188, 'gamma': 0.024710524294828975}
Train classification report for patient 30 with window 288:
              precision    recall  f1-score   support

         0.0       0.86      0.83      0.85     95257
         1.0       0.84      0.86      0.85     95257

    accuracy                           0.85    190514
   macro avg       0.85      0.85      0.85    190514
weighted avg       0.85      0.85      0.85    190514

Validation classification report for patient 30 with window 288:
              precision    recall  f1-score   support

         0.0       0.95      0.73      0.83     12674
         1.0       0.11      0.47      0.17       864

    accuracy                           0.72     13538
   macro avg       0.53      0.60      0.50     13538
weighted avg       0.90      0.72      0.79     13538

Test classification report for patient 30 with window 288:
              precision    recall  f1-score   support

         0.0     

/usr/local/lib/python3.10/dist-packages/imblearn/over_sampling/_smote/base.py:336: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(
[I 2024-07-23 14:16:45,153] A new study created in memory with name: no-name-39c6ea57-acbb-4ba9-8c6e-c52a74092e7e


13539
13539
13419
13419


[I 2024-07-23 14:16:59,069] Trial 0 finished with value: 0.6070816516876221 and parameters: {'C': 0.0007459343285726547, 'gamma': 0.566984951147885}. Best is trial 0 with value: 0.6070816516876221.
[I 2024-07-23 14:17:13,145] Trial 1 finished with value: 0.5294944047927856 and parameters: {'C': 0.0457056309980145, 'gamma': 0.009846738873614562}. Best is trial 0 with value: 0.6070816516876221.
[I 2024-07-23 14:17:27,469] Trial 2 finished with value: 0.2789279818534851 and parameters: {'C': 6.0268891286825045e-05, 'gamma': 6.025215736203858e-05}. Best is trial 0 with value: 0.6070816516876221.
[I 2024-07-23 14:17:41,862] Trial 3 finished with value: 0.6177378296852112 and parameters: {'C': 1.951722464144947e-05, 'gamma': 0.21423021757741043}. Best is trial 3 with value: 0.6177378296852112.
[I 2024-07-23 14:17:54,659] Trial 4 finished with value: 0.573401927947998 and parameters: {'C': 0.010129197956845729, 'gamma': 0.034702669886504146}. Best is trial 3 with value: 0.6177378296852112.
[I

Best parameters found by Optuna: {'C': 0.8178061863266235, 'gamma': 0.13246744228820476}
Train classification report for patient 30 with window 288:
              precision    recall  f1-score   support

         0.0       0.99      0.94      0.97     95353
         1.0       0.95      0.99      0.97     95353

    accuracy                           0.97    190706
   macro avg       0.97      0.97      0.97    190706
weighted avg       0.97      0.97      0.97    190706

Validation classification report for patient 30 with window 288:
              precision    recall  f1-score   support

         0.0       0.95      0.88      0.91     12674
         1.0       0.16      0.33      0.21       864

    accuracy                           0.85     13538
   macro avg       0.55      0.60      0.56     13538
weighted avg       0.90      0.85      0.87     13538

Test classification report for patient 30 with window 288:
              precision    recall  f1-score   support

         0.0      